## Getting started with the GEOStudio

To download this notebook click [here.](./GeospatialStudio-First-Steps.ipynb)


You should have a clean deployment of the studio before running this notebook. If you havent deployed locally yet, navigate to [here](https://github.com/Beldine-Moturi/AMLD-Africa-2026) for a guide on how to deploy, or you can follow along a more detailed version of the guide in the [GEOStudio repository](https://github.com/terrastackai/geospatial-studio).

The steps below will enable you to onboard some initial artefacts, before trying out the functionality.

#### 1. Navigate to the UI front page and create an api key. Click on the Manage your API keys link. This should pop-up a window where you can generate, access and delete your api keys. UI link: [https://localhost:4180](https://localhost:4180)


#### 2. Copy your new api key and the Studio API URL and paste them in the cell below to save it as an environment variable

In [ ]:
%cd geospatial-studio/

In [ ]:
%env STUDIO_API_KEY="<your api key from the UI>"`

In [ ]:
%env BASE_GATEWAY_API_URL=https://localhost:4181

#### 3. Onboard the sandbox-models, these are placeholder models (pipelines) for onboarding existing inferences or testing tuned models.

In [ ]:
!./deployment-scripts/add-sandbox-models.sh

#### 4. Onboard an existing inference output (useful for loading examples)

Onboard one of the inferences. This will start a pipeline to pull the data and set it up in the platform. You should now be able to browser to the inferences page in the UI and view the example/s you have added.

In [ ]:
!python populate-studio/populate-studio.py inferences
# select "AGB Data - Karen, Nairobi,kenya"

#### 5. Onboard an existing tuned models and run inference

We will onboard a tuned model from a URL. This is initiated by an API call, which will trigger the onboarding process, starting download in the backend. Once the download is completed, it should appear with completed status in the UI models/tunes page. First we ensure we have a tuning task templates.
Onboard the tuning task templates. These are the outline configurations to make basic tuning tasks easier for users.

In [ ]:
!python populate-studio/populate-studio.py templates
# select  1. Segmentation - Generic template v1 and v2 models: Segmentation

In [ ]:
!python populate-studio/populate-studio.py tunes
# select "prithvi-eo-flood - prithvi-eo-flood"

#### 6. Test with an inference run

Now we can trigger an inference run. This can be run through the UI or API (as here), where you tell which spatial and temporal domain over which to run inference. You need to get the tune_id for the onboarded tune (from the onboarding response or from the models/tunes page in the UI) and paste it into the command below.

In [ ]:
tune_id=<paste tune_id here>

payload='{
"model_display_name": "geofm-sandbox-models",
"location": "Red Bluff, California, United States",
"description": "Park Fire Aug 2024",
"spatial_domain": {
  "bbox": [],
  "urls": [
    "https://geospatial-studio-example-data.s3.us-east.cloud-object-storage.appdomain.cloud/examples-for-inference/park_fire_scaled.tif"
  ],
  "tiles": [],
  "polygons": []
},
"temporal_domain": [
  "2024-08-12"
],
"pipeline_steps": [
  {
    "status": "READY",
    "process_id": "url-connector",
    "step_number": 0
  },
  {
    "status": "WAITING",
    "process_id": "terratorch-inference",
    "step_number": 1
  },
  {
    "status": "WAITING",
    "process_id": "postprocess-generic",
    "step_number": 2
  },
  {
    "status": "WAITING",
    "process_id": "push-to-geoserver",
    "step_number": 3
  }
],
"post_processing": {
  "cloud_masking": "False",
  "ocean_masking": "False",
  "snow_ice_masking": null,
  "permanent_water_masking": "False"
},
"model_input_data_spec": [
  {
    "bands": [
      {
        "index": "0",
        "RGB_band": "B",
        "band_name": "Blue",
        "scaling_factor": "0.0001"
      },
      {
        "index": "1",
        "RGB_band": "G",
        "band_name": "Green",
        "scaling_factor": "0.0001"
      },
      {
        "index": "2",
        "RGB_band": "R",
        "band_name": "Red",
        "scaling_factor": "0.0001"
      },
      {
        "index": "3",
        "band_name": "NIR_Narrow",
        "scaling_factor": "0.0001"
      },
      {
        "index": "4",
        "band_name": "SWIR1",
        "scaling_factor": "0.0001"
      },
      {
        "index": "5",
        "band_name": "SWIR2",
        "scaling_factor": "0.0001"
      }
    ],
    "connector": "sentinelhub",
    "collection": "hls_l30",
    "file_suffix": "_merged.tif",
    "modality_tag": "HLS_L30"
  }
],
"geoserver_push": [
  {
    "z_index": 0,
    "workspace": "geofm",
    "layer_name": "input_rgb",
    "file_suffix": "",
    "display_name": "Input image (RGB)",
    "filepath_key": "model_input_original_image_rgb",
    "geoserver_style": {
      "rgb": [
        {
          "label": "RedChannel",
          "channel": 1,
          "maxValue": 255,
          "minValue": 0
        },
        {
          "label": "GreenChannel",
          "channel": 2,
          "maxValue": 255,
          "minValue": 0
        },
        {
          "label": "BlueChannel",
          "channel": 3,
          "maxValue": 255,
          "minValue": 0
        }
      ]
    },
    "visible_by_default": "True"
  },
  {
    "z_index": 1,
    "workspace": "geofm",
    "layer_name": "pred",
    "file_suffix": "",
    "display_name": "Model prediction",
    "filepath_key": "model_output_image",
    "geoserver_style": {
      "segmentation": [
        {
          "color": "#000000",
          "label": "ignore",
          "opacity": 0,
          "quantity": "-1"
        },
        {
          "color": "#000000",
          "label": "no-data",
          "opacity": 0,
          "quantity": "0"
        },
        {
          "color": "#ab4f4f",
          "label": "fire-scar",
          "opacity": 1,
          "quantity": "1"
        }
      ]
    },
    "visible_by_default": "True"
  }
 ]
}'

echo $payload | curl -X POST "${UI_ROUTE_URL}/studio-gateway/v2/tunes/${tune_id}/try-out" \
  --header 'Content-Type: application/json' \
  --header "X-API-Key: $STUDIO_API_KEY" \
  --insecure \
  --data @-




